In [ ]:
''' 1. Module Import '''
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, datasets
''' 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.11.0+cu113  Device: cuda


In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

class CustomDataset(Dataset):
	def __init__(self, csv_path):
		df = pd.read_csv(csv_path)
		
		self.inp = df.iloc[:, :3].values
		self.outp = df.iloc[:,-1].values.reshape(-1,1)
	
	def __len__(self):
		# 가지고 있는 데이터셋의 길이를 반환한다.
		return len(self.inp) # 1314

	def __getitem__(self,idx):
		inp = torch.FloatTensor(self.inp[idx])
		outp = torch.LongTensor(self.outp[idx])
		return inp, outp # 해당하는 idx(인덱스)의 input과 output 데이터를 반환한다.

''' 3. MNIST 데이터 다운로드 (Train set, Test set 분리하기) '''
PATH_TRAIN = './drive/MyDrive/WELT/data_set_train.csv'
PATH_TEST = './drive/MyDrive/WELT/data_set_test.csv'

train_dataset = CustomDataset(PATH_TRAIN)
test_dataset = CustomDataset(PATH_TEST)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

In [ ]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 3]) type: torch.FloatTensor
y_train: torch.Size([32, 1]) type: torch.LongTensor


In [ ]:
for (X_test, y_test) in test_loader:
    print('X_test:', X_train.size(), 'type:', X_train.type())
    print('y_test:', y_train.size(), 'type:', y_train.type())
    break

X_test: torch.Size([32, 3]) type: torch.FloatTensor
y_test: torch.Size([32, 1]) type: torch.LongTensor


In [ ]:
''' 6. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.dropout_prob = 0.5
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)

    def forward(self, x):
        x = x.view(-1, 3)
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = F.dropout(x, training = self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [ ]:
''' 7. Optimizer, Objective Function 설정하기 '''
import torch.nn.init as init
def weight_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight.data)

model = Net().to(DEVICE)
model.apply(weight_init)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

print(model)

Net(
  (fc1): Linear(in_features=3, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=9, bias=True)
  (batch_norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [ ]:
''' 8. MLP 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (feature, label) in enumerate(train_loader):
        feature = feature.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(feature)
        loss = criterion(output, label.squeeze(dim=-1))
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(feature), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
''' 9. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for feature, label in test_loader:
            feature = feature.to(DEVICE)
            label = label.to(DEVICE)
            output = model(feature)
            test_loss += criterion(output, label.squeeze(dim=-1)).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
''' 10. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/27288 (0%)]	Train Loss: 2.810086
Train Epoch: 1 [6400/27288 (23%)]	Train Loss: 0.892870
Train Epoch: 1 [12800/27288 (47%)]	Train Loss: 0.862045
Train Epoch: 1 [19200/27288 (70%)]	Train Loss: 0.701074
Train Epoch: 1 [25600/27288 (94%)]	Train Loss: 1.150988

[EPOCH: 1], 	Test Loss: 0.6235, 	Test Accuracy: 73.98 % 

Train Epoch: 2 [0/27288 (0%)]	Train Loss: 0.887027
Train Epoch: 2 [6400/27288 (23%)]	Train Loss: 0.947695
Train Epoch: 2 [12800/27288 (47%)]	Train Loss: 0.439550
Train Epoch: 2 [19200/27288 (70%)]	Train Loss: 0.859310
Train Epoch: 2 [25600/27288 (94%)]	Train Loss: 0.748310

[EPOCH: 2], 	Test Loss: 0.5816, 	Test Accuracy: 80.27 % 

Train Epoch: 3 [0/27288 (0%)]	Train Loss: 0.556183
Train Epoch: 3 [6400/27288 (23%)]	Train Loss: 1.254200
Train Epoch: 3 [12800/27288 (47%)]	Train Loss: 1.272072
Train Epoch: 3 [19200/27288 (70%)]	Train Loss: 0.798873
Train Epoch: 3 [25600/27288 (94%)]	Train Loss: 0.818539

[EPOCH: 3], 	Test Loss: 0.5789, 	Test Accuracy: 79.49 % 

T